In [1]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
torch.cuda.is_available()

2.0.1
0.14.1+cu116


/home/hsiao-ching/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


True

In [4]:
# import torch
from pipeline.mplug_owl.modeling_mplug_owl import MplugOwlForConditionalGeneration
from pipeline.mplug_owl.tokenization_mplug_owl import MplugOwlTokenizer
from pipeline.mplug_owl.processing_mplug_owl import MplugOwlImageProcessor, MplugOwlProcessor

model = torch.load('/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/mPLUG_model.pth')
pretrained_ckpt = 'MAGAer13/mplug-owl-llama-7b'

# model = MplugOwlForConditionalGeneration.from_pretrained(
#     pretrained_ckpt,
#     torch_dtype=torch.bfloat16,
# )
image_processor = MplugOwlImageProcessor.from_pretrained(pretrained_ckpt)
tokenizer = MplugOwlTokenizer.from_pretrained(pretrained_ckpt)
processor = MplugOwlProcessor(image_processor, tokenizer)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'MplugOwlTokenizer'.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

MplugOwlForConditionalGeneration(
  (vision_model): MplugOwlVisionModel(
    (embeddings): MplugOwlVisionEmbeddings(
      (patch_embed): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (pre_layernorm): LayerNormFp32((1024,), eps=1e-06, elementwise_affine=True)
    )
    (encoder): MplugOwlVisionEncoder(
      (layers): ModuleList(
        (0): MplugOwlVisionEncoderLayer(
          (self_attn): MplugOwlVisionAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (input_layernorm): LayerNormFp32((1024,), eps=1e-06, elementwise_affine=True)
          (mlp): MplugOwlMLP(
            (activation_fn): QuickGELU()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
      

In [4]:
# We use a human/AI template to organize the context as a multi-turn conversation.
# <image> denotes an image placehold.
prompts = [
'''The following is a conversation between a curious human and AI assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
Human: <image>
Human: What is or What are the object(s) in the photo? What are the functions this/these object(s) featured? Please list the components that can form this/these object(s) in detail.
AI: ''']

# The image paths should be placed in the image_list and kept in the same order as in the prompts.
# We support urls, local file paths and base64 string. You can custom the pre-process of images by modifying the mplug_owl.modeling_mplug_owl.ImageProcessor
#image_list = ['/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/assembly.png']

In [10]:
# generate kwargs (the same in transformers) can be passed in the do_generate()
generate_kwargs = {
    'do_sample': True,
    'top_k': 25,
    'max_length': 512
}
from PIL import Image
images = [Image.open(_) for _ in image_list]
inputs = processor(text=prompts, images=images, return_tensors='pt')
inputs = {k: v.bfloat16() if v.dtype == torch.float else v for k, v in inputs.items()}
inputs = {k: v.to(model.device) for k, v in inputs.items()}
with torch.no_grad():
    res = model.generate(**inputs, **generate_kwargs)
sentence = tokenizer.decode(res.tolist()[0], skip_special_tokens=True)
print(sentence)

The object in the photo is a microphone. It has a gray exterior and a circular shape. The microphone has four small dots or legs surrounding it. This object can perform several functions, such as recording audio signals, amplifying voices, capturing sounds for playback in various media formats, and for transmitting audio signals over the internet or other communication channels. The four dots, also known as 'legs' or 'spikes', are designed to enhance stability when placed on a surface, which is crucial for accurate audio recording or playback. Additionally, these legs may serve as an attachment point for a support structure, such as a microphone stand, or be used with other microphone accessories, such as a microphone cable.


In [5]:
def pipeline_of_mPLUG_Owl(image_list, prompt, generate_kwargs):

    # mPLUG-Owl
    images = [Image.open(_) for _ in image_list]
    inputs = processor(text=prompts, images=images, return_tensors='pt')
    inputs = {k: v.bfloat16() if v.dtype == torch.float else v for k, v in inputs.items()}
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        res = model.generate(**inputs, **generate_kwargs)
    sentence = tokenizer.decode(res.tolist()[0], skip_special_tokens=True)
    
    
    
    # Store the text into the file that the corresponding image located
    image_str = ' '.join(image_list)
    folder = image_str.replace("assembly.png", "") + "mPLUG_Owl.txt"
    
    with open(folder, 'w') as file:
        file.write(sentence)


In [14]:
# generate kwargs (the same in transformers) can be passed in the do_generate()
generate_kwargs = {
    'do_sample': True,
    'top_k': 25,
    'max_length': 512
}
from PIL import Image

test_file = ["/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/assembly.png"]
pipeline_of_mPLUG_Owl(test_file, "Please list the possible components that may compose the objects in this image, and provide a detailed description, dividing them into main components and sub-components. The more detailed, the better.", generate_kwargs)

In [11]:
# Process all assembly files in Dataset
import os
import time
from tqdm import tqdm, trange
assemblies = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset")

for assembly in tqdm(assemblies):
        try:
                files = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly)
                for file in files:
                        pipeline_of_mPLUG_Owl(["/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly+"/"+file + "/assembly.png"], "What is or What are the object(s) in the photo? What are the functions this/these object(s) featured? Please list the components that can form this/these object(s) in detail.", generate_kwargs)
        except: 
                pass     

100%|██████████| 11/11 [8:05:01<00:00, 2645.57s/it] 


In [12]:
# Count the number of folders in Assembly Dataset_all
import os
import time
from tqdm import tqdm, trange

assemblies = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset")

for assembly in tqdm(assemblies):
    # 初始化資料夾計數器
    folder_count = 0

    # 設定要檢查的資料夾路徑
    folder_path = "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly

    # 使用 os.walk() 函式遍歷目錄下的所有檔案和資料夾
    for _, dirs, _ in os.walk(folder_path):
        # 更新資料夾計數器
        folder_count += len(dirs)
    print(f"目錄{assembly}中的資料夾數量為: {folder_count}")


  9%|▉         | 1/11 [00:08<01:27,  8.74s/it]

目錄Assembly Dataset_0中的資料夾數量為: 751


 18%|█▊        | 2/11 [00:17<01:21,  9.02s/it]

目錄Assembly Dataset_1中的資料夾數量為: 751


 27%|██▋       | 3/11 [00:29<01:22, 10.36s/it]

...

目錄Assembly Dataset_8中的資料夾數量為: 751


 91%|█████████ | 10/11 [01:19<00:07,  7.01s/it]

目錄Assembly Dataset_9中的資料夾數量為: 751


100%|██████████| 11/11 [01:27<00:00,  7.96s/it]

目錄Assembly Dataset_10中的資料夾數量為: 741


In [13]:
# Count the number of folders in Assembly Dataset_all
import os
import time
from tqdm import tqdm, trange

assemblies = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset")

for assembly in tqdm(assemblies):
    # 初始化資料夾計數器
    folder_count = 0
    files = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly)

    for file in files:
        
        # 設定要檢查的資料夾路徑
        folder_path = "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly+"/"+file

        if os.path.exists(os.path.join(folder_path, "assembly.png")):
            folder_count += 1
        
    print(f"目錄{assembly}中的assembly.png數量為: {folder_count}")


 18%|█▊        | 2/11 [00:00<00:00,  9.26it/s]

目錄Assembly Dataset_0中的assembly.png數量為: 751
目錄Assembly Dataset_1中的assembly.png數量為: 751


 36%|███▋      | 4/11 [00:00<00:00,  9.18it/s]

目錄Assembly Dataset_2中的assembly.png數量為: 751
目錄Assembly Dataset_3中的assembly.png數量為: 751


 64%|██████▎   | 7/11 [00:00<00:00, 10.83it/s]

目錄Assembly Dataset_4中的assembly.png數量為: 751
目錄Assembly Dataset_5中的assembly.png數量為: 751
目錄Assembly Dataset_6中的assembly.png數量為: 751


 82%|████████▏ | 9/11 [00:00<00:00, 10.58it/s]

目錄Assembly Dataset_7中的assembly.png數量為: 751
目錄Assembly Dataset_8中的assembly.png數量為: 751


100%|██████████| 11/11 [00:01<00:00,  9.96it/s]

目錄Assembly Dataset_9中的assembly.png數量為: 751
目錄Assembly Dataset_10中的assembly.png數量為: 741


In [23]:
import os
folder_path = "/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl"
files = os.listdir(folder_path)
image_paths = []
for file in files:
        if file.endswith(".png") and not(file.endswith("assembly.png")):  # 確認檔案是否為.png檔
            image_path = os.path.join(folder_path, file)
            image_paths.append(image_path)
        elif file.endswith("assembly.png"):
            image = os.path.join(folder_path, file)
        elif file.endswith("mPLUG_Owl.txt"):
            txt_path = os.path.join(folder_path,file)
            f = open(txt_path, 'r')
            description = f.read()
            f.close()
image_answer = ", ".join([image_path for image_path in image_paths])          
print(image_paths)
print(image)
print(description)
print(image_answer)
print(type(image_answer))

['/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/images.png', '/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/cookiemonster.png']
/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/assembly.png
There is an object in the photo that can be described as either a mic for Skype, an FM recorder, or both. This device serves as the audio input and transmitter for computer-based communication systems like Skype and FM systems. It comes with a cable connected to a PC or Laptop, which allows the user to record audio and send it through a communication medium like Skype or any audio transmission system. There are also a few other objects in the photo, but their specific function is not apparent. It is likely that the additional objects are part of the setup for using the mic or the FM recorder system.
/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/images.png, /home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/cookiemonster.png
<class 'str'>


In [ ]:
# 處理所有資料夾中的.png檔案並輸出jsonl檔案
import os
import time
import json
from tqdm import tqdm, trange
assembly_path = "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset"
assemblies = os.listdir(assembly_path)

for assembly in tqdm(assemblies):
    try:
        folder_path = os.path.join(assembly_path, assembly) # "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_0"
        folders = os.listdir(folder_path)
        for folder in folders:
            assembly_file = os.path.join(folder_path, folder)
            print(assembly_file)
    except:
        pass

### Arrange the whole assembly dataset to a .jsonl format
<p>Inspired by the OwlEval Dataset, we sort out our assembly dataset which consist of lots of components images and the description generated by mPLUG-Owl into a .jsonl format.</p>
For example: 
<code>{"image": "/path/to/whole image", "description": "The object in the photo is a black metal rod. ...", "answer": "/path/to/part 1", "/path/to /part 2",...}</code>

In [24]:
import os
import time
import json
from tqdm import tqdm, trange

# 定義處理函式
def process_folder(folder_path, output_file):
    files = os.listdir(folder_path)
    image_paths = []
    for file in files:
        if file.endswith(".png") and not(file.endswith("assembly.png")):
            image_path = os.path.join(folder_path, file)
            image_paths.append(image_path)
        elif file.endswith("assembly.png"):
            image = os.path.join(folder_path, file)
        elif file.endswith("mPLUG_Owl.txt"):
            txt_path = os.path.join(folder_path,file)
            f = open(txt_path, 'r')
            image_description = f.read()
            f.close()
    
    image_answer = ", ".join([image_path for image_path in image_paths])
    
    with open(output_file, 'a') as f:
        data = {
            "image": image,  # Whole assembly image
            "description": image_description,
            "answer": image_answer # assembly parts
        }
        f.write(json.dumps(data) + '\n')


# 設定輸出檔案路徑
output_jsonl = "/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/train.jsonl"
# 處理所有資料夾中的.png檔案並輸出jsonl檔案
assembly_path = "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset"
assemblies = os.listdir(assembly_path)

for assembly in tqdm(assemblies):
    try:
        folder_path = os.path.join(assembly_path, assembly) # "/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_0"
        folders = os.listdir(folder_path)
        for folder in folders:
            assembly_file = os.path.join(folder_path, folder)
            process_folder(assembly_file, output_jsonl)
    except Exception as e:
        print(e)


 18%|█▊        | 2/11 [01:24<06:49, 45.46s/it]

[Errno 20] Not a directory: '/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_1/Fusion 360 Gallery Dataset Public License.docx'


 27%|██▋       | 3/11 [02:02<05:35, 41.97s/it]

[Errno 20] Not a directory: '/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_2/Fusion 360 Gallery Dataset Public License.docx'


 36%|███▋      | 4/11 [02:45<04:55, 42.28s/it]

...

[Errno 20] Not a directory: '/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_8/Fusion 360 Gallery Dataset Public License.docx'


 91%|█████████ | 10/11 [20:30<01:46, 106.57s/it]

[Errno 20] Not a directory: '/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_9/Fusion 360 Gallery Dataset Public License.docx'


100%|██████████| 11/11 [25:06<00:00, 136.93s/it]

[Errno 20] Not a directory: '/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/Assembly Dataset_10/Fusion 360 Gallery Dataset Public License.docx'


In [ ]:
# Turn all assembly data into jsonl format

# Origin training data format:
''' {"image": ["xxx.jpg"], "text": "The following is a conversation between a curious human and AI assistant. The assistant gives helpful, detailed, 
        and polite answers to the user's questions.\nHuman: <image>\nHuman: Where is the man located in the image?\nAI: The man is located in a small restaurant, 
        sitting and enjoying his meal.\nHuman: What type of food is the man eating in the image?\nAI: The man is eating a burrito in the image.
        \nHuman: What can be seen next to the man in the restaurant?\nAI: Next to the man in the restaurant, there is a fridge filled with cans of soda.
        \nHuman: How does the man feel as he is about to eat his meal?\nAI: The man looks happy and is smiling as he is about to take a bite of his meal.
        \nHuman: What unique aspect of the photo makes the scene appear distorted or unusual?\nAI: The unique aspect of this photo is that it was taken using a fish-eye lens, 
        which creates a distorted, wide-angle view of the scene, making it appear different from a standard perspective.", "task_type": "llava_sft"}
''' 

# Our format:
'''{"image": ["xxx.png"], "text": "The following is a conversation between a curious human and AI assistant. The assistant gives helpful, detailed, and polite answers 
        to the user's questions.\nHuman: <image>\nHuman: What is or What are the object(s) in the photo? What are the functions this/these object(s) featured? Please 
        list the components that can form this/these object(s) in detail.\nAI: <images_emb>",  
        "task_type": "llava_sft"}
'''

import os
import time
from tqdm import tqdm, trange
assemblies = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset") #./Assembly Dataset_0
folder_jsonl = "/home/hsiao-ching/Desktop/palm-e/mPLUG-Owl/configs/train.yaml"

for assembly in tqdm(assemblies):
        try:
                files = os.listdir("/media/hsiao-ching/ADATA HV320/PaLM-E Experiments/Fusion 360 Assembly Data/Assembly Dataset/"+assembly)
                for file in files:
                        
        except: 
                pass     